# Urban Sound Classification 

In [1]:
import sys
print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


In [2]:
import tensorflow as tf
tf.__version__

'1.12.0'

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) #gpu 확인

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15166277912288398436
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7663451028479636953
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13276675287482182307
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10619178189
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 2
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 3
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 4
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 5
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 6

In [4]:
from keras.utils import to_categorical 
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, AveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend.tensorflow_backend as K
from sklearn.model_selection import train_test_split
import numpy as np
import os


Using TensorFlow backend.


In [5]:
npz_files = np.load('./urban8K_STFT_All-002.npz')

In [6]:
npz_files.files

['X', 'y', 'fn_s']

In [7]:
X = npz_files['X']
y = npz_files['y']
fn_s = npz_files['fn_s']

In [8]:
X.shape, y.shape 

((7333, 89010), (7333,))

In [9]:
y[0], fn_s[0]

(8.0, 'Data/UrbanSound8K/audio/fold10/159742-8-0-2.wav')

In [10]:
y = to_categorical(y)

In [11]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [12]:
shuffle_index = np.random.permutation(len(X))
X = X[shuffle_index]
y = y[shuffle_index]
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5866, 89010), (5866, 10), (1467, 89010), (1467, 10))

In [14]:
X_train = X_train.reshape(X_train.shape[0],129,690,1)
X_test = X_test.reshape(X_test.shape[0],129,690,1)

In [15]:
X_train.shape, X_test.shape

((5866, 129, 690, 1), (1467, 129, 690, 1))

**sequential()**

version1

In [16]:
def CNN(input_shape) :
        
        model = Sequential()
        
        model.add(Conv2D(32, kernel_size=(5,5), strides=(1,1), padding='same',
                         activation='relu', input_shape=input_shape))

        model.add(MaxPooling2D(pool_size=(4,8), strides=(4,8)))

        model.add(Conv2D(64, kernel_size=(5,5), strides=(1,1), padding='same',
                         activation='relu'))

        model.add(MaxPooling2D(pool_size=(4,8), strides=(4,8)))

        model.add(Flatten())

        model.add(Dense(512, activation='relu'))

        model.add(Dropout(0.5))

        model.add(Dense(10, activation='softmax'))

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        return model 

In [17]:
image_size = (129,690,1)
model = CNN(image_size)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 129, 690, 32)      832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 86, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 86, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 10, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5120)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2621952   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
__________

In [18]:
pat = 5 
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)


MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
model_path = MODEL_SAVE_FOLDER_PATH + 'urban_sound1.h5'
model_checkpoint = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)

epochs = 20
batch_size = 100
input_shape = (129,690,1)
model = CNN(input_shape)
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, model_checkpoint], 
              verbose=1, validation_split=0.1)  

Train on 5279 samples, validate on 587 samples
Epoch 1/20
5279/5279 [==============================] - 14s 3ms/step - loss: 1.7477 - acc: 0.3950 - val_loss: 1.3046 - val_acc: 0.5315

Epoch 00001: val_loss improved from inf to 1.30461, saving model to ./model/urban_sound1.h5
Epoch 2/20
5279/5279 [==============================] - 10s 2ms/step - loss: 1.2280 - acc: 0.5829 - val_loss: 1.0451 - val_acc: 0.6320

Epoch 00002: val_loss improved from 1.30461 to 1.04511, saving model to ./model/urban_sound1.h5
Epoch 3/20
5279/5279 [==============================] - 11s 2ms/step - loss: 1.0162 - acc: 0.6492 - val_loss: 0.9872 - val_acc: 0.6542

Epoch 00003: val_loss improved from 1.04511 to 0.98725, saving model to ./model/urban_sound1.h5
Epoch 4/20
5279/5279 [==============================] - 11s 2ms/step - loss: 0.8575 - acc: 0.7088 - val_loss: 0.8431 - val_acc: 0.7394

Epoch 00004: val_loss improved from 0.98725 to 0.84311, saving model to ./model/urban_sound1.h5
Epoch 5/20
5279/5279 [=======

In [19]:
model = load_model('./model/urban_sound1.h5')
model.evaluate(X_test, y_test)

1467/1467 [==============================] - 2s 1ms/step


[0.555438765434481, 0.8568507162746152]

version2

In [20]:
def CNN2(input_shape) :

        model = Sequential()

        model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same',
                             activation='relu', input_shape=input_shape))

        model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same',
                             activation='relu'))

        model.add(MaxPooling2D(pool_size=(3,3), strides=(3,3), padding='valid'))

        model.add(Dropout(0.25))

        model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same',
                             activation='relu'))

        model.add(Conv2D(64, kernel_size=(3,5), strides=(2,3), padding='same',
                             activation='relu'))

        model.add(MaxPooling2D(pool_size=(3,3), strides=(3,3), padding='same'))

        model.add(Conv2D(64, kernel_size=(10,3), strides=(3,3), padding='same',
                             activation='relu'))

        model.add(AveragePooling2D(pool_size=(3,3), strides=(2,3), padding='same'))

        model.add(Dropout(0.25))

        model.add(Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same',
                             activation='relu'))

        model.add(Flatten())

        model.add(Dense(1024, activation='relu'))

        model.add(Dropout(0.5))

        model.add(Dense(10, activation='softmax'))
        
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



        return model 


In [21]:
input_shape = (129,690,1)
model = CNN2(input_shape)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 129, 690, 32)      320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 129, 690, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 43, 230, 32)       0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 43, 230, 32)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 43, 230, 64)       18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 22, 77, 64)        61504     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 26, 64)         0         
__________

In [22]:
epochs = 20
batch_size = 100

pat = 5
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
model_path = MODEL_SAVE_FOLDER_PATH + 'urban_sound2.h5'
model_checkpoint = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, model_checkpoint], verbose=1, validation_split=0.1)  

Train on 5279 samples, validate on 587 samples
Epoch 1/20
5279/5279 [==============================] - 23s 4ms/step - loss: 1.8889 - acc: 0.3082 - val_loss: 1.5573 - val_acc: 0.4617

Epoch 00001: val_loss improved from inf to 1.55726, saving model to ./model/urban_sound2.h5
Epoch 2/20
5279/5279 [==============================] - 19s 4ms/step - loss: 1.3863 - acc: 0.5007 - val_loss: 1.1930 - val_acc: 0.5520

Epoch 00002: val_loss improved from 1.55726 to 1.19297, saving model to ./model/urban_sound2.h5
Epoch 3/20
5279/5279 [==============================] - 20s 4ms/step - loss: 1.1130 - acc: 0.6206 - val_loss: 1.0001 - val_acc: 0.6235

Epoch 00003: val_loss improved from 1.19297 to 1.00009, saving model to ./model/urban_sound2.h5
Epoch 4/20
5279/5279 [==============================] - 21s 4ms/step - loss: 0.9162 - acc: 0.6818 - val_loss: 0.9024 - val_acc: 0.6610

Epoch 00004: val_loss improved from 1.00009 to 0.90244, saving model to ./model/urban_sound2.h5
Epoch 5/20
5279/5279 [=======

In [23]:
model = load_model('./model/urban_sound2.h5')
model.evaluate(X_test, y_test)

1467/1467 [==============================] - 3s 2ms/step


[0.3525529625205337, 0.9154737556801414]

**functional**

version1

In [24]:
# Convolutional Neural Network
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Concatenate
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D


visible = Input(shape=(129,690,1))

conv1 = Conv2D(32, kernel_size=(5, 5), strides=(1,1), activation='relu', padding='same')(visible)

pool1 = MaxPooling2D(pool_size=(4, 8), strides=(4,8))(conv1)

conv2 = Conv2D(64, kernel_size=(5, 5), strides=(1,1), activation='relu', padding='same')(pool1)

pool2 = MaxPooling2D(pool_size=(4, 8), strides=(4,8))(conv2)

flat = Flatten()(pool2)

hidden1 = Dense(512, activation='relu')(flat)

output = Dense(10, activation='softmax')(hidden1)

model = Model(inputs=visible, outputs=output)

print(model.summary())

# plot graph
# plot_model(model, to_file='convolutional_neural_network.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 129, 690, 1)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 129, 690, 32)      832       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 86, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 86, 64)        51264     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 10, 64)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 5120)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               2621952   
__________

In [25]:
epochs = 20
batch_size = 100

pat = 5
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
model_path = MODEL_SAVE_FOLDER_PATH + 'urban_sound1_functional.h5'
model_checkpoint = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, model_checkpoint], verbose=1, validation_split=0.1)  

Train on 5279 samples, validate on 587 samples
Epoch 1/20
5279/5279 [==============================] - 12s 2ms/step - loss: 1.6368 - acc: 0.4467 - val_loss: 1.2269 - val_acc: 0.5656

Epoch 00001: val_loss improved from inf to 1.22692, saving model to ./model/urban_sound1_functional.h5
Epoch 2/20
5279/5279 [==============================] - 12s 2ms/step - loss: 1.0782 - acc: 0.6437 - val_loss: 0.9710 - val_acc: 0.6951

Epoch 00002: val_loss improved from 1.22692 to 0.97095, saving model to ./model/urban_sound1_functional.h5
Epoch 3/20
5279/5279 [==============================] - 11s 2ms/step - loss: 0.8206 - acc: 0.7384 - val_loss: 0.8751 - val_acc: 0.7342

Epoch 00003: val_loss improved from 0.97095 to 0.87510, saving model to ./model/urban_sound1_functional.h5
Epoch 4/20
5279/5279 [==============================] - 11s 2ms/step - loss: 0.6368 - acc: 0.7979 - val_loss: 0.7620 - val_acc: 0.7649

Epoch 00004: val_loss improved from 0.87510 to 0.76201, saving model to ./model/urban_sound1

In [26]:
model = load_model('./model/urban_sound1_functional.h5')
model.evaluate(X_test, y_test)

1467/1467 [==============================] - 2s 1ms/step


[0.7074067416082211, 0.8145875942568919]

version2

In [27]:
visible = Input(shape=(129,690,1))

conv1 = Conv2D(32, kernel_size=(3, 3), strides=(1,1), activation='relu', padding='same')(visible)

conv2 = Conv2D(32, kernel_size=(3, 3), strides=(1,1), activation='relu', padding='same')(conv1)

pool2 = MaxPooling2D(pool_size=(3, 3), strides=(3,3))(conv2)

drop2 = Dropout(0.25)(pool2)

conv3 = Conv2D(64, kernel_size=(3, 3), strides=(1,1), activation='relu', padding='same')(drop2)

conv4 = Conv2D(64, kernel_size=(3, 5), strides=(2,3), activation='relu', padding='same')(conv3)

pool4 = MaxPooling2D(pool_size=(3, 3), strides=(3,3))(conv4)

conv5 = Conv2D(64, kernel_size=(10, 3), strides=(3,3), activation='relu', padding='same')(pool4)

pool5 = AveragePooling2D(pool_size=(3, 3), strides=(2,3))(conv4)

mrg = Concatenate(axis=1)([pool4, pool5]) 

drop3 = Dropout(0.25)(mrg)

conv6 = Conv2D(128, kernel_size=(3, 3), strides=(1,1), activation='relu', padding='same')(drop3)

flat = Flatten()(conv6)

fc = Dense(1024, activation='relu')(flat)

output = Dense(10, activation='softmax')(fc)

model = Model(inputs=visible, outputs=output)

print( model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 129, 690, 1)  0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 129, 690, 32) 320         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 129, 690, 32) 9248        conv2d_13[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, 43, 230, 32)  0           conv2d_14[0][0]                  
__________________________________________________________________________________________________
dropout_6 

In [28]:
epochs = 20
batch_size = 100

pat = 5
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
model_path = MODEL_SAVE_FOLDER_PATH + 'urban_sound2_functional.h5'
model_checkpoint = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, model_checkpoint], verbose=1, validation_split=0.1)  

Train on 5279 samples, validate on 587 samples
Epoch 1/20
5279/5279 [==============================] - 26s 5ms/step - loss: 1.8360 - acc: 0.3493 - val_loss: 1.4189 - val_acc: 0.4906

Epoch 00001: val_loss improved from inf to 1.41892, saving model to ./model/urban_sound2_functional.h5
Epoch 2/20
5279/5279 [==============================] - 24s 4ms/step - loss: 1.2549 - acc: 0.5618 - val_loss: 1.2188 - val_acc: 0.5724

Epoch 00002: val_loss improved from 1.41892 to 1.21881, saving model to ./model/urban_sound2_functional.h5
Epoch 3/20
5279/5279 [==============================] - 23s 4ms/step - loss: 0.9818 - acc: 0.6653 - val_loss: 1.0338 - val_acc: 0.6508

Epoch 00003: val_loss improved from 1.21881 to 1.03381, saving model to ./model/urban_sound2_functional.h5
Epoch 4/20
5279/5279 [==============================] - 23s 4ms/step - loss: 0.7483 - acc: 0.7420 - val_loss: 0.9099 - val_acc: 0.7189

Epoch 00004: val_loss improved from 1.03381 to 0.90986, saving model to ./model/urban_sound2

In [29]:
model = load_model('./model/urban_sound2_functional.h5')
model.evaluate(X_test, y_test)

1467/1467 [==============================] - 3s 2ms/step


[0.7069239361845236, 0.8323108385676986]